# 3. Clustering
- Word2Vec
- greedy_modularity_communities 클러스터 수행

## 1. Word2Vec

In [2]:
# Library
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [7]:
# 여기서부터 실행했을 경우에만 실행
token_df = pd.read_csv('./Data/Preprocess_Data/word_token.csv')

In [9]:
# 복사본
t = token_df.copy()

In [10]:
# year 칼럼의 뒤의 두 자리 추출
t['year_suffix'] = t['year'].astype(str).str[-2:]
# list 칼럼의 단어들을 분리하여 리스트로 변환
word_lists = t['tokens'].str.split()

In [12]:
new_word_lists = []
for words, suffix in tqdm(zip(word_lists, t['year_suffix']), total=len(t)):
    new_words = [str(suffix) + '_' + word for word in words]
    new_word_lists.append(new_words)

100%|███████████████████████████████| 2267163/2267163 [07:46<00:00, 4859.48it/s]


In [14]:
# 리스트를 하나의 문자열로 변환하여 list 칼럼에 덮어씌움
t['tokens'] = pd.Series([' '.join(words) for words in tqdm(new_word_lists)])

# year_suffix 칼럼 제거
t = t.drop('year_suffix', axis=1)

100%|██████████████████████████████| 2267163/2267163 [00:26<00:00, 84862.57it/s]


In [16]:
t.head()

,tokens,year
0,13_method 13_peer_to_peer_streaming 13_video_o...,2013
1,13_paper 13_discus 13_bacterial 13_network_com...,2013
2,13_article 13_treat 13_digital_humanity 13_wor...,2013
3,13_work 13_describes 13_preliminary 13_step 13...,2013
4,13_goal 13_extraction 13_learning_by_demonstra...,2013


In [ ]:
t.to_csv('Data/Preprocess_Data/word_token+year.csv', encoding='utf-8-sig', index=False)

### 연도 집어넣은 것을 바탕으로 word2vec 시행

In [3]:
t = pd.read_csv('./Data/Preprocess_Data/word_token+year.csv')

In [4]:
t.head()

,tokens,year
0,13_method 13_peer_to_peer_streaming 13_video_o...,2013
1,13_paper 13_discus 13_bacterial 13_network_com...,2013
2,13_article 13_treat 13_digital_humanity 13_wor...,2013
3,13_work 13_describes 13_preliminary 13_step 13...,2013
4,13_goal 13_extraction 13_learning_by_demonstra...,2013


In [5]:
# 텍스트 데이터 전처리 및 토큰화, 불용어 제거
t['tokens2'] = t['tokens'].apply(lambda x: [word for word in word_tokenize(x)])
sentences = t['tokens2'].tolist()

KeyboardInterrupt: 

In [ ]:
sentences

In [ ]:
# Word2Vec 모델 학습
model = Word2Vec(sentences, min_count=1)

# 단어 벡터 확인
word_vectors = model.wv

In [20]:
# 모델 저장
model.save('Data/Preprocess_Data/year_word2vec.model')

In [19]:
# 단어 벡터 저장
word_vectors = model.wv
word_vectors.save("Data/Preprocess_Data/year_word_vectors.kv")

In [27]:
similar_words = model.wv.most_similar('20_deep_learning', topn=30)
print(similar_words)

[('20_deep_learning_such', 0.9240278005599976), ('20_face_tracking', 0.922843337059021), ('20_self_calibration', 0.9179024696350098), ('20_batch_processing', 0.916452169418335), ('20_quantum_annealing', 0.9142048954963684), ('20_sw', 0.9131624698638916), ('20_tasnet', 0.9126613736152649), ('20_parallel_structure', 0.9123446345329285), ('20_biped', 0.9123086333274841), ('20_tsv', 0.9116710424423218), ('20_broadens', 0.9115983843803406), ('20_matrix_profile', 0.9092435240745544), ('20_duo', 0.9091187119483948), ('20_coral', 0.9090192317962646), ('20_semantic_slam', 0.9086829423904419), ('20_pea', 0.9064937233924866), ('20_graphic_processing_unit_in', 0.9060552716255188), ('20_hierarchal', 0.9059861302375793), ('20_defect_prediction', 0.9052213430404663), ('20_supercomputing_system', 0.9050830602645874), ('20_convolutional_neural_network_used', 0.9048026204109192), ('20_system_form', 0.903627872467041), ('20_deep_learning_on', 0.9034767150878906), ('20_tmp', 0.9033932089805603), ('20_qsm'

In [38]:
model.wv.similarity('13_machine_learning', '14_machine_learning')

0.4937513

In [24]:
# 2020년 데이터만
t_2023 = t[t['year']==2023]
t_2023.reset_index(drop=True, inplace=True)

In [ ]:
t_2023.head()

In [ ]:
# 텍스트 데이터 전처리 및 토큰화
t_2023['tokens2'] = t_2023['tokens'].astype(str).apply(lambda x: word_tokenize(x))
t_2023['tokens2'] = t_2023['tokens2'].apply(lambda tokens: [token for token in tokens if token != ','])


sentences = t_2023['tokens2'].tolist()
sentences = [sentence for sentence in tqdm(sentences, desc="Processing sentences")]

In [ ]:
sentences

In [1]:
# sentences_flat에서 19년도 단어가 들어간 데이터만 찾기
sentences_flat = [word for sentence in sentences for word in sentence]

SyntaxError: invalid syntax (1144936424.py, line 1)

In [ ]:
# 주어진 단어들에 대한 Word2Vec 결과 벡터 추출
word_vecs = [word_vectors[word] for word in sentences_flat]

# 모든 단어 쌍의 코사인 유사도 계산
similarity_matrix = cosine_similarity(word_vecs)

# 결과를 DataFrame으로 변환
similarity_df = pd.DataFrame(similarity_matrix, index=sentences_flat, columns=btq_19)

In [ ]:
# 유사성 임계값 설정
similarity_threshold = 0.13

# 주어진 단어들에 대한 Word2Vec 결과 벡터 추출
word_vecs = [word_vectors[word] for word in sentences_flat if word in word_vectors]

# 각 커뮤니티의 중심점을 찾기
centroids = []
for community in communities:
    community_vectors = [word_vectors[word] for word in community if word in word_vectors]
    if len(community_vectors) > 0:
        community_centroid = sum(community_vectors) / len(community_vectors)
        centroids.append(community_centroid)

# 커뮤니티 중심점을 기반으로 새 그래프 생성
G_centroids = nx.Graph()

# 중심점 노드 추가
for i, centroid in enumerate(community_centroids):
    G_centroids.add_node(centroid, label=f"Community {i+1}")

# 코사인 유사도를 사용하여 중심점 간의 유사도 계산 및 연결
G_centroids = nx.Graph()
for i in range(len(centroids)):
    for j in range(i+1, len(centroids)):
        centroid_i = centroids[i]
        centroid_j = centroids[j]
        similarity = cosine_similarity([centroid_i], [centroid_j])[0][0]
        if similarity > similarity_threshold:
            G_centroids.add_edge(f"Centroid {i+1}", f"Centroid {j+1}", weight=similarity)

# 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 7))
node_labels = nx.get_node_attributes(G_centroids, 'label')
nx.draw(G_centroids, labels=node_labels, ax=ax)
plt.title('Paper Abstract Network with Communities')
plt.show()